# Exercise 2 - The Data Producer Perspective
## Dataset Description
In this exercise we want to experience the effort it takes to provide a good description of a dataset as the data producer. To achieve that, we are looking at how our example dataset "Order Positions" is generated. Following that, we ask you to first provide a schema definition for our dataset and aftewards provide a free text description of the same. Please keep in mind all the issues we faced and discussed before and how the schema or the description could help pointing those out.
## Dataset Generation
The two cells below show the relevant core functions for the dataset generation. If you would like to understand the full context, feel free to execute the load function below to see the full code of the data generator.

In [1]:
#generation of single non-empty order line
def generate_order_line(order_id, line_number):
  article_id = random.choice(WEIGHTED_ARTICLE_IDS) #each bought article is a random choice factoring in the previously added weights
  article_name = ARTICLES[article_id][0] #reference article name from the input dataset
  article_description = ARTICLES[article_id][1] #reference article description from the input dataset
  article_group = add_article_group(article_id) #reference article group from the input dataset, can be randomly empty in 5% of cases
  original_price = ARTICLE_ORIGINAL_PRICES[article_id] #reference article original price, as previously generated
  discounted_price = round(original_price*random.choice(DISCOUNT_WEIGHTS),2) #actual price factoring in a random discount
  earnings_after_taxes = round(discounted_price*0.81, 2) #earnings after deducting 19% taxes
  order_line = [[
    str(order_id),
    str(line_number),
    str(article_id),
    article_name,
    article_description,
    article_group,
    str(original_price),
    str(discounted_price),
    str(earnings_after_taxes)]]
  return order_line

In [2]:
#generation of given number of orders
def generate_orders(number_of_orders):
  order_positions = []
  for order_id in range(1, number_of_orders+1):
    line_numbers = random.choice(LINE_NUMBER_COUNT_WEIGHTS) #randomly select between 0 and 5 items per order. Orders can be empty!
    if line_numbers == 0: #handling of empty orders
      order_positions += [[str(order_id), "", "", "null", "null", "null", "0.00", "0.00", "0.00"]]
    else:
      for line_number in range(1, line_numbers+1):
        order_positions += generate_order_line(order_id, line_number)
  return order_positions

In [3]:
#if you would like to see the full code, feel free to run:
load ../data/data_gen.py

## Provide a Dataset Schema


In [4]:
Order_positions(
  order_id integer not null,                   # 0-5 items per order
  line_number integer nullable,                # orders can be empty
  article_id integer nullable,                 # orders can be empty
  article_name string not null,                # can be string "null"
  article_description string not null,         # can be string "null"
  article_group string nullable,               # group not always provided
  original_price Decimal(35,2) not null,       # 0.00 for empty orders
  discounted_price Decimal(35,2) not null,     # 0.00 for empty orders
  earnings_after_taxes Decimal(35,2) not null  # 0.00 for empty orders
)


## Provide a Dataset Description
Please use the space below to write a free text dataset description.

The Order_positions dataset contains all information related to the sales made through our platform. For each order placed by our customers multiple records are created that each represent one item bought within that order. A single record can always be identified by its order_id and a line_number, its position within that order. Please note, if all content of an order is returned by the customer, a reference to the order is kept through a record with the order_id, but empty remaining attributes.
For each ordered item you will find additional article information. An article_id is kept as a reference that identifies the article. It is followed by an article_name, article_descriptions, and article_group. Please note, the upstream service that provides this information is not fully reliable which can lead to empty article_group values even for known articles.
The dataset contains three different price values. The first one is the original_price of the article as listed in the shop, the second is the discounted_price which represents the actual sales value after applying an order specific discount. The third attribute contains the earnings_after_taxes which is the discounted_price minus 19% VAT.